#Logistic Regression

1. **Setup and Data Loading**

**Imports**:

- LogisticRegression from scikit-learn for modeling.

- StandardScaler for feature standardization (critical for gradient-based optimization).

- classification_report for evaluation.

**Data Loading**:

- Loads magic04.data with particle collision features.

- Converts class labels to binary (1=Gamma, 0=Hadron).

In [ ]:
# Setup and Data Loading
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression


In [ ]:
# Load data
cols = ["fLength", "fWidth", "fSize", "fConc", "fConcl", "fAsym",
        "fM3Long", "fM3Trans", "fAlpha", "fDist", "class"]
df = pd.read_csv("magic04.data", names=cols)
df["class"] = (df["class"] == "g").astype(int)  # Binary: 1=Gamma, 0=Hadron

2. **Data Preparation**

- **prepare_data() Function**:

- - Standardizes features (StandardScaler) to ensure equal weighting in the logistic loss function.

- - Oversampling: Balances classes using RandomOverSampler to mitigate bias toward the majority class (Hadrons).

- **Train-Test Split**:

- - 60% training, 20% validation, 20% test (shuffled).

In [ ]:
# Data Preparation
def prepare_data(df, oversample=False):
    X = df[cols[:-1]].values
    y = df[cols[-1]].values
    X = StandardScaler().fit_transform(X)  # Standardize features
    if oversample:
        from imblearn.over_sampling import RandomOverSampler
        X, y = RandomOverSampler().fit_resample(X, y)
    return X, y

In [ ]:
# Split data
train, valid, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])
X_train, y_train = prepare_data(train, oversample=True)  # Balance classes
X_test, y_test = prepare_data(test)

3. **Training and Evaluation**

- **Model Initialization**:
 - lr = LogisticRegression()  # Default: L2 regularization

- **Training**:
 - lr.fit(X_train, y_train)  # Learns coefficients (weights) via optimization

- **Prediction & Evaluation**:
 - y_pred = lr.predict(X_test)
 - print(classification_report(y_test, y_pred))

Reports precision, recall, F1-score, and accuracy.

In [ ]:
# Train and Evaluate Logistic Regression
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred))

4. **Coefficients Analysis**

- **Feature Importance**:
 - plt.bar(cols[:-1], lr.coef_[0])  # Plot coefficients
 - Positive weights: Increase the probability of class 1 (Gamma).
 - Negative weights: Decrease the probability of class 1.

In [ ]:
# Coefficients Analysis
plt.bar(cols[:-1], lr.coef_[0])  # Feature importance

5. **Decision Boundary Visualization (2D PCA)**

- **PCA Reduction**: Projects features onto 2 principal components for plotting.

- **Meshgrid Prediction**: Generates a grid of points and predicts their class using the trained model.

- **Plot**:

- - plt.contourf(xx, yy, Z, alpha=0.3)  # Decision boundary
- - plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train)  # Data points

Shows how the linear boundary separates classes in reduced space.

In [ ]:
# Visualize Decision Boundary (2D PCA Projection)
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import LogisticRegression


# Step 1: Reduce features to 2D using PCA
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Step 2: Retrain KNN on 2D data (for visualization)
#knn_pca = KNeighborsClassifier(n_neighbors=5)
#knn_pca.fit(X_train_pca, y_train)
model_pca = LogisticRegression().fit(X_train_pca, y_train)

# Step 3: Create a meshgrid for decision boundary
x_min, x_max = X_train_pca[:, 0].min() - 1, X_train_pca[:, 0].max() + 1
y_min, y_max = X_train_pca[:, 1].min() - 1, X_train_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                     np.arange(y_min, y_max, 0.02))
Z = model_pca.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Step 4: Plot decision boundary and data points
plt.figure(figsize=(10, 6))
plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.Paired)
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train,
            edgecolor='k', s=50, cmap=plt.cm.Paired)
plt.title("Logistic Regression Decision Boundary (PCA-Reduced Data)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.colorbar(label="Class (0=Hadron, 1=Gamma)")
plt.show()